In [1]:
import torch
from experiments.sol_propagator import PDESolutionPropogator
from experiments.eqn_sol import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## test propogateHeat

In [7]:
x = torch.rand(10, 20, 4, dtype=torch.float).to(device)
A = scipy.sparse.random(10, 10, density=0.1, format='csc', dtype=np.float32)
A = A + A.T
A = A.toarray()
A = A + np.diag(np.ones(A.shape[0]))
d = A.sum(axis=0)
D = np.diag(d)
Dinvhf = np.diag(1/np.sqrt(d))
laplacian = Dinvhf @ (D - A) @ Dinvhf

sol = HeatEqnSolSparseExpm(laplacian, device).to(device)
y1 = sol(x, 3)

oper = PDESolutionPropogator(laplacian, device, eps=1e-5)
y2 = oper.propogateHeat(x, torch.linspace(3, 4, 13))

torch.abs(y1 - y2[0]).max().item()

6.013613600974743e-07

## test propogateWave

In [8]:
sol = WaveEqnSol(laplacian, device, eps=1e-5).to(device)
y1 = sol(x, x, 4.)

y2 = oper.propogateWave(x, x, torch.linspace(4, 5, 13))

torch.abs(y1 - y2[0]).max().item()

4.76837158203125e-07